In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
cardboard = []
for i in range(1,404):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/cardboard/cardboard"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(227,227))
    X = np.array(temp)
    cardboard.append(X)
cardboard = np.array(cardboard)
cardboard = np.take(cardboard,np.random.permutation(cardboard.shape[0]),axis=0)
print(cardboard.shape)

In [ ]:
glass = []
for i in range(1,502):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/glass/glass"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(227,227))
    X = np.array(temp)
    glass.append(X)
glass = np.array(glass)
glass = np.take(glass,np.random.permutation(glass.shape[0]),axis=0)
print(glass.shape)

In [ ]:
metal = []
for i in range(1,411):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/metal/metal"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(227,227))
    X = np.array(temp)
    metal.append(X)
metal = np.array(metal)
metal = np.take(metal,np.random.permutation(metal.shape[0]),axis=0)
print(metal.shape)

In [ ]:
paper = []
for i in range(1,595):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/paper/paper"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(227,227))
    X = np.array(temp)
    paper.append(X)
paper = np.array(paper)
paper = np.take(paper,np.random.permutation(paper.shape[0]),axis=0)
print(paper.shape)

In [ ]:
plastic = []
for i in range(1,483):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/plastic/plastic"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(227,227))
    X = np.array(temp)
    plastic.append(X)
plastic = np.array(plastic)
plastic = np.take(plastic,np.random.permutation(plastic.shape[0]),axis=0)
print(plastic.shape)

In [ ]:
trash = []
for i in range(1,138):
    temp = tf.keras.preprocessing.image.load_img(
        path = "trashnet/trash/trash"+str(i)+".jpg",
        grayscale=False, color_mode='rgb',target_size=(227,227))
    X = np.array(temp)
    trash.append(X)
trash = np.array(trash)
trash = np.take(trash,np.random.permutation(trash.shape[0]),axis=0)
print(trash.shape)

In [ ]:
train_X = np.vstack((cardboard[:302,:,:,:],glass[:376,:,:,:],metal[:307,:,:,:],
                    paper[:445,:,:,:],plastic[:361,:,:,:],trash[:103,:,:,:]))
test_X = np.vstack((cardboard[302:,:,:,:],glass[376:,:,:,:],metal[307:,:,:,:],
                   paper[445:,:,:,:],plastic[361:,:,:,:],trash[103:,:,:,:]))
train_Y = np.zeros((6,train_X.shape[0]))
test_Y = np.zeros((6,test_X.shape[0]))
for i in range(train_X.shape[0]):
    if(i<302):
        train_Y[0][i] = 1
    elif(i<678):
        train_Y[1][i] = 1
    elif(i<985):
        train_Y[2][i] = 1
    elif(i<1430):
        train_Y[3][i] = 1
    elif(i<1791):
        train_Y[4][i] = 1
    else:
        train_Y[5][i] = 1
for i in range(test_X.shape[0]):
    if(i<101):
        test_Y[0][i] = 1
    elif(i<226):
        test_Y[1][i] = 1
    elif(i<329):
        test_Y[2][i] = 1
    elif(i<478):
        test_Y[3][i] = 1
    elif(i<599):
        test_Y[4][i] = 1
    else:
        test_Y[5][i] = 1

In [ ]:
test_X_re = test_X
test_Y_re = test_Y

In [ ]:
np.random.seed(1013)
train_perm = np.random.permutation(train_X.shape[0])
train_X = np.take(train_X,train_perm,axis=0)
print(train_X.shape)
train_Y = np.take(train_Y,train_perm,axis=1)
print(train_Y.shape)
np.random.seed(901)
test_perm = np.random.permutation(test_X.shape[0])
test_X = np.take(test_X,test_perm,axis=0)
print(test_X.shape)
test_Y = np.take(test_Y,test_perm,axis=1)
print(test_Y.shape)

In [ ]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,height_shift_range=0.1, rotation_range=45, zoom_range=0.1,
    horizontal_flip=True,vertical_flip=True)
train_gen.fit(train_X)

In [ ]:
def AlexNetCE(input_shape = (227, 227, 3), classes = 6):
    X_input = tf.keras.Input(input_shape)
    X = X_input
    X = tf.keras.layers.Conv2D(96, (11, 11), strides = (4, 4), activation = "relu", name = 'conv1')(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = tf.keras.layers.Conv2D(256, (5, 5), padding = "same",activation = "relu", name = 'conv2')(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_conv2')(X)
    X = tf.keras.layers.Conv2D(256, (3, 3), padding = "same",activation = "relu", name = 'conv5')(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(4096, activation = "relu", name='fc' + str(1))(X)
    X = tf.keras.layers.Dense(4096, activation = "relu", name='fc' + str(2))(X)
    X = tf.keras.layers.Dense(classes, activation='softmax', name='fc' + str(classes))(X)
       
    # Create model
    model = tf.keras.Model(inputs = X_input, outputs = X, name='ALEXNETCE')

    return model

In [ ]:
model = AlexNetCE()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.000005), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
train_loss = []
train_acc = []
test_loss = []
test_acc = []
for i in range(20):
    print("Training Process "+str(len(train_loss)))
    history = model.fit(train_gen.flow(train_X, train_Y.T, batch_size=32),steps_per_epoch=60,epochs=1)
    print("Test Result "+str(len(train_loss)))
    preds = model.evaluate(test_X, test_Y.T)
    print ("Loss = " + str(preds[0]))
    print ("Test Accuracy = " + str(preds[1]))
    train_loss.append(history.history["loss"][0])
    train_acc.append(history.history["accuracy"][0])
    test_loss.append(preds[0])
    test_acc.append(preds[1])
plt.plot(train_loss)
plt.show()
plt.plot(test_loss)
plt.show()

In [ ]:
for i in range(10):
    print("Training Process "+str(len(train_loss)))
    history = model.fit(train_gen.flow(train_X, train_Y.T, batch_size=32),steps_per_epoch=60,epochs=1)
    print("Test Result "+str(len(train_loss)))
    preds = model.evaluate(test_X, test_Y.T)
    print ("Loss = " + str(preds[0]))
    print ("Test Accuracy = " + str(preds[1]))
    train_loss.append(history.history["loss"][0])
    train_acc.append(history.history["accuracy"][0])
    test_loss.append(preds[0])
    test_acc.append(preds[1])
plt.plot(train_loss)
plt.show()
plt.plot(test_loss)
plt.show()

In [ ]:
for i in range(10):
    print("Training Process "+str(len(train_loss)))
    history = model.fit(train_X, train_Y.T,batch_size=32,epochs=1)
    print("Test Result "+str(len(train_loss)))
    preds = model.evaluate(test_X, test_Y.T)
    print ("Loss = " + str(preds[0]))
    print ("Test Accuracy = " + str(preds[1]))
    train_loss.append(history.history["loss"][0])
    train_acc.append(history.history["accuracy"][0])
    test_loss.append(preds[0])
    test_acc.append(preds[1])
plt.plot(train_loss)
plt.show()
plt.plot(test_loss)
plt.show()

In [ ]:
for i in range(10):
    print("Training Process "+str(len(train_loss)))
    history = model.fit(train_X, train_Y.T,batch_size=32,epochs=1)
    print("Test Result "+str(len(train_loss)))
    preds = model.evaluate(test_X, test_Y.T)
    print ("Loss = " + str(preds[0]))
    print ("Test Accuracy = " + str(preds[1]))
    train_loss.append(history.history["loss"][0])
    train_acc.append(history.history["accuracy"][0])
    test_loss.append(preds[0])
    test_acc.append(preds[1])
plt.plot(train_loss)
plt.show()
plt.plot(test_loss)
plt.show()

In [ ]:
for i in range(10):
    print("Training Process "+str(len(train_loss)))
    history = model.fit(train_X, train_Y.T,batch_size=32,epochs=1)
    print("Test Result "+str(len(train_loss)))
    preds = model.evaluate(test_X, test_Y.T)
    print ("Loss = " + str(preds[0]))
    print ("Test Accuracy = " + str(preds[1]))
    train_loss.append(history.history["loss"][0])
    train_acc.append(history.history["accuracy"][0])
    test_loss.append(preds[0])
    test_acc.append(preds[1])
plt.plot(train_loss)
plt.show()
plt.plot(test_loss)
plt.show()

In [ ]:
error = [0,0,0,0,0,0]
mislabel = np.zeros((6,6))
prediction = model.predict(test_X_re)
for i in range(test_X_re.shape[0]):
    if(test_Y_re[np.argmax(prediction,axis=1)[i],i] == 0):
        if(i<101):
            error[0] = error[0]+1
            mislabel[0][np.argmax(prediction,axis=1)[i]] += 1
        elif(i<226):
            error[1] = error[1]+1
            mislabel[1][np.argmax(prediction,axis=1)[i]] += 1
        elif(i<329):
            error[2] = error[2]+1
            mislabel[2][np.argmax(prediction,axis=1)[i]] += 1
        elif(i<478):
            error[3] = error[3]+1
            mislabel[3][np.argmax(prediction,axis=1)[i]] += 1
        elif(i<599):
            error[4] = error[4]+1
            mislabel[4][np.argmax(prediction,axis=1)[i]] += 1
        else:
            error[5] = error[5]+1
            mislabel[5][np.argmax(prediction,axis=1)[i]] += 1

In [ ]:
total = np.array([101,125,103,149,121,34])
error = np.array(error)
error = np.divide(error,total)
print("% of mislabeled cardboard: "+str(error[0]))
print("Mostly mislabeled as: "+str(np.argmax(mislabel,axis=0)[0]))
print("% of mislabeled glass: "+str(error[1]))
print("Mostly mislabeled as: "+str(np.argmax(mislabel,axis=0)[1]))
print("% of mislabeled metal: "+str(error[2]))
print("Mostly mislabeled as: "+str(np.argmax(mislabel,axis=0)[2]))
print("% of mislabeled paper: "+str(error[3]))
print("Mostly mislabeled as: "+str(np.argmax(mislabel,axis=0)[3]))
print("% of mislabeled plastic: "+str(error[4]))
print("Mostly mislabeled as: "+str(np.argmax(mislabel,axis=0)[4]))
print("% of mislabeled trash: "+str(error[5]))
print("Mostly mislabeled as: "+str(np.argmax(mislabel,axis=0)[5]))

In [ ]:
#f = open('train_loss_aug_p.txt', 'w')
#for i in train_loss:
#    f.write(str(i))
#    f.write("\n")
#f.close()

In [ ]:
#f = open('test_loss_aug_p.txt', 'w')
#for i in test_loss:
#    f.write(str(i))
#    f.write("\n")
#f.close()

In [ ]:
#f = open('train_acc_aug_p.txt', 'w')
#for i in train_acc:
#    f.write(str(i))
#    f.write("\n")
#f.close()

In [ ]:
#f = open('test_acc_aug_p.txt', 'w')
#for i in test_acc:
#    f.write(str(i))
#    f.write("\n")
#f.close()

In [ ]:
#f = open('error_aug.txt', 'w')
#for i in error:
#    f.write(str(i))
#    f.write("\n")
#f.close()